In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
train.shape , test.shape

# Data Fields
* datetime - hourly date + timestamp  
* season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
* 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
* 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
* 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* count - number of total rentals

In [ ]:
train.head()

In [ ]:
test.head()          

test에는 casual , registred , count가 없음.

In [ ]:
submission.head()

In [ ]:
train.info()
print('-'*50)
test.info()

In [ ]:
import missingno as msno
msno.matrix(train, figsize=(10,5))

null값은 tarin, test 둘다 없고  데이터 타입도 같음.

split으로 문자열 자르기 \
lambda 함수 dataframe축 적용기능\
apply 함수 : dataframe의 데이터를 일괄 가공해줌. \
datetime 내장모듈과  피쳐와 이름이 같음으로 혼돈 주의

In [ ]:
print(train['datetime'][50]) # 50번째 날짜출력
print(train['datetime'][50].split()) #공백기준으로 나누기
print(train['datetime'][50].split()[0]) #날짜출력
print(train['datetime'][50].split()[1])  #시간출력

In [ ]:
print(train['datetime'][50].split()[0])   #날짜 출력
print(train['datetime'][50].split()[0].split('-'))  # '-' 기준으로 문자열 나누기
print(train['datetime'][50].split()[0].split('-')[0]) # 년도
print(train['datetime'][50].split()[0].split('-')[1]) #월
print(train['datetime'][50].split()[0].split('-')[2]) #일

In [ ]:
print(train['datetime'][50].split()[1])   #시간출력
print(train['datetime'][50].split()[1].split(':'))  #":" 기준으로 나누기
print(train['datetime'][50].split()[1].split(':')[0])  #시간
print(train['datetime'][50].split()[1].split(':')[1]) #분
print(train['datetime'][50].split()[1].split(':')[2]) #초

In [ ]:
train['date'] = train['datetime'].apply(lambda x: x.split()[0])  #날짜 피쳐를 생성

train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])  #연도생성
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1]) #달
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])   #일
train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])  #시간
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1]) #분
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2]) #초

datetime과 calendar 모듈을 이용해서 날짜,문자열 추출

In [ ]:
from datetime import datetime 
import calendar

print(train['datetime'][50]) # 50번째 날짜출력
print(datetime.strptime(train['date'][50], '%Y-%m-%d'))   #datetime 타입으로 변경
print(datetime.strptime(train['date'][50], '%Y-%m-%d').weekday())   #정수로 요일 반환   0-월요일 1-화요일 2-수요일 3-목요일 ....
print(calendar.day_name[datetime.strptime(train['date'][50], '%Y-%m-%d').weekday()])  #문자열로 요일 반환

# 모델훈련할땐 피처값을 문자열로 변환x,  모두다 숫자로 변환해야함

In [ ]:
# 요일(weekday)에 피처를 추가
train['weekday'] = train['date'].apply(
    lambda dateString:
    calendar.day_name[datetime.strptime(dateString,'%Y-%m-%d').weekday()])


In [ ]:
# season , weather 피처 추가 및 매핑

train['season'] = train['season'].map({1: 'Spring',
                                       2: 'Summer',
                                       3: 'Fall',
                                       4: 'Winter' })

train['weather'] = train['weather'].map({1: 'Clear',
                                       2: 'Mist, Few clouds',
                                       3: 'Light Snow, Rain, Thunderstorm',
                                       4: 'Heavy Rain, Thunderstorm, Snow, Fog' })


In [ ]:
train.head()

#date 컬럼 제거예정
# month 3달 =  1 seasson 이니까 month 제거

In [ ]:
# 시각화
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mpl.rc('font', size=12)  #폰트크기 12
sns.displot(train['count']);  #분포도 출력

회귀 모델이 좋은 성능을 내려면 데이터가 정규분포를 따라야하는데 분포가 왼쪽으로 많이 편향되어 있음.\
따라서 데이터를 정규분포에 가깝게 만들기위해 로그변환을 이용

In [ ]:
sns.displot(np.log(train['count']));

In [ ]:
# 막대 그래프로 시각화
mpl.rc('font', size =10)  #폰트 크기 조절
mpl.rc('axes', titlesize =15) #제목 크기 조절
figure, axes = plt.subplots(nrows=3, ncols=2)    #3행 2열 형태의 figure 표
plt.tight_layout()   #그래프 사이에 여백 
figure.set_size_inches(10,9)  #전체 figure크기 설정

# x,y축에 수량 막대 그래프 생성
sns.barplot(x='year', y='count', data=train, ax=axes[0,0])
sns.barplot(x='month', y='count', data=train, ax=axes[0,1])
sns.barplot(x='day', y='count', data=train, ax=axes[1,0])
sns.barplot(x='hour', y='count', data=train, ax=axes[1,1])
sns.barplot(x='minute', y='count', data=train, ax=axes[2,0])
sns.barplot(x='second', y='count', data=train, ax=axes[2,1])

#서브플롯에 제목 입력
axes[0,0].set(title='Rental amounts by year')
axes[0,1].set(title='Rental amounts by month')
axes[1,0].set(title='Rental amounts by day')
axes[1,1].set(title='Rental amounts by hour')
axes[2,0].set(title='Rental amounts by minute')
axes[2,1].set(title='Rental amounts by second')

# x축 라벨 회전
# axes[1,0].tick_params(axis='x', labelrotation=90)
# axes[1,1].tick_params(axis='x', labelrotation=90)

3번째 그래프 day 가 1~19일까지만 존재.   day 피쳐 제거\
5,6번 그래프  minute , second  피쳐도 제거

In [ ]:
axes

In [ ]:
#박스플롯 은 범주형 데이터에 따른 수치형 데이터 정보를 나타내는 그래프 , 범주형 데이터에 따라 대여 수량이 어떻게 변화하는지 보기 편함
figure,axes = plt.subplots(nrows=2, ncols=2)
plt.tight_layout()
figure.set_size_inches(10,10)

#박스플롯 생성
sns.boxplot(x='season', y='count', data=train, ax=axes[0,0])
sns.boxplot(x='weather', y='count', data=train, ax=axes[0,1])
sns.boxplot(x='holiday', y='count', data=train, ax=axes[1,0])     # 1은 공휴일 , 0은 공휴일 x 
sns.boxplot(x='workingday', y='count', data=train, ax=axes[1,1]) 

# 제목입력
axes[0,0].set(title='box plot on count across Seasson')
axes[0,1].set(title='box plot on count across Weather')
axes[1,0].set(title='box plot on count across Holiday')
axes[1,1].set(title='box plot on count across Working day')

# 0,1 x축 라벨 겹치는거 해결
axes[0,1].tick_params(axis='x', labelrotation=8)


In [ ]:
# 포인트플롯은 범주형 데이터에 따른 수치형 데이터의 평균과 신뢰구간을 점과 선으로 표시
mpl.rc('font', size=10)
figure, axes = plt.subplots(nrows=5)
figure.set_size_inches(12,20)

sns.pointplot(x='hour', y='count', data=train, hue='workingday', ax=axes[0])
sns.pointplot(x='hour', y='count', data=train, hue='holiday', ax=axes[1])
sns.pointplot(x='hour', y='count', data=train, hue='weekday', ax=axes[2])
sns.pointplot(x='hour', y='count', data=train, hue='season', ax=axes[3])
sns.pointplot(x='hour', y='count', data=train, hue='weather', ax=axes[4])

weather ==4인 데이터 제거해서 처리하기

In [ ]:
# 회귀선을 포함한 산점도 그래프는 수치형 데이터 간 상관관계를 파악하는데 사용

mpl.rc('font', size=12)
figure, axes= plt.subplots(nrows=2, ncols=2)
plt.tight_layout()
figure.set_size_inches(8,6)

#수량 산점도 그래프
sns.regplot(x='temp', y='count', data=train, ax=axes[0,0], scatter_kws={'alpha':0.3}, line_kws={'color':'blue'})
sns.regplot(x='atemp', y='count', data=train, ax=axes[0,1], scatter_kws={'alpha':0.3}, line_kws={'color':'blue'})
sns.regplot(x='windspeed', y='count', data=train, ax=axes[1,0], scatter_kws={'alpha':0.3}, line_kws={'color':'blue'})
sns.regplot(x='humidity', y='count', data=train, ax=axes[0,1], scatter_kws={'alpha':0.3}, line_kws={'color':'blue'})

#scatter_kws = 산점도 그래프에 찍히는 점의 투명도
#line_kws = 회귀선 색상


windspeed 의 결측치가 많음. 풍속이 0일때 데이터값이 많음. 따라서 피처 제거해야함

In [ ]:
#히트맵   수치형 데이터끼리의 상관관계 파악
# temp, atemp, humidity , windspeed, count는 수치형 데이터. 
train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()

In [ ]:
corrMat= train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()
fig, ax = plt.subplots()
fig.set_size_inches(10,10)
sns.heatmap(corrMat, annot=True)  #상관관계 숫자로 표시   1에 가까울수록 관계많음.
ax.set(title='heatmap of numerical data')

모델링 계획\
가장 기본적인 회귀모델, 릿지 , 라쏘 , 랜덤포레스트 이용\
피처 엔지니어링 : 동일하게 사용\
하이퍼파라미터 최적화 : 그리드서치 이용\
타깃값이 count가 아닌 log(count)임을 기억

In [ ]:
import pandas as pd

train = pd.read_csv('../input/bike-sharing-demand/train.csv')
test = pd.read_csv('../input/bike-sharing-demand/test.csv')
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
#weather 이상치 데이터 제거
train = train[train['weather'] !=4]

In [ ]:
all_data_temp = pd.concat([train,test])
all_data_temp

In [ ]:
all_data = pd.concat([train,test], ignore_index=True)  # 결측치 제거후 남은 인덱스를 무시하고 이어붙임
all_data

In [ ]:
# day , minute, second 피쳐 제거
from datetime import datetime

all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])  #날짜 피쳐를 생성

all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])  #연도생성
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1]) #달

all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])  #시간

all_data['weekday'] = all_data['date'].apply(
    lambda dateString:
    datetime.strptime(dateString,'%Y-%m-%d').weekday())

In [ ]:
#불필요한 피쳐 제거
drop_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month']
all_data = all_data.drop(drop_features, axis=1)

In [ ]:
#데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]   #타깃값이 있으면 훈련데이터
X_test = all_data[pd.isnull(all_data['count'])]  # 없으면 테스트 데이터

X_train.head()

In [ ]:
# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count']

X_train.head()

# 평가지표 계산함수 작성

https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FkOfso%2FbtqIKrVmDus%2FqqEWAdPMkQm2nANZCKsSu1%2Fimg.png

In [ ]:
# RMSLE공식을 넘파이 그대로 구현한 코드
import numpy as np

def rmsle(y_true, y_pred, covertExp = True):
    #지수변환
    if covertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    output = np.sqrt(np.mean((log_true - log_pred) **2 ))
    return output


# model learning

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

In [ ]:
log_y = np.log(y) #타깃값 로그변환
linear_reg_model.fit(X_train, log_y)

In [ ]:
# preds = linear_reg_model(X_train)

# print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y, preds, True):.4f}')

In [ ]:
linearreg_preds = linear_reg_model.predict(X_test)  #테스트 데이터로 예측

submission['count'] = np.exp(linearreg_preds) #지수변환
submission.to_csv('submission_linear.csv', index = False) #파일저장


# ridge model -규제를 적용한 선형 회귀 모델 
규제란 ? : 모델이 훈련데이터에 과대적합이 되ㅣ지 않도록 해주는 방법 

GridSearch - 가장 기본적인 하이퍼 파라미터 최적화 기법, 정해진 하이퍼파라미터를 모두 탐색하며 가장 좋은 성능을 내는 값을 찾는 방식. 단점으로 오래걸림 \
하이퍼파라미터 - 사용자가 직접 셋팅하는 설정값

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_model = Ridge()

In [ ]:
# 하이퍼파라미터 값 목록
ridge_params ={'max_iter':[3000], 'alpha': [0.1, 1, 2, 3, 4, 10, 30 ,100, 200, 300, 400, 800, 900, 1000]}

#교차 검증용 평가 수행 RMSLE점수 계산
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 with Ridge 객체 생성
gridsearch_ridge_model = GridSearchCV(estimator = ridge_model,    #릿지모델
                                     param_grid = ridge_params,   #값 목록
                                     scoring = rmsle_scorer,     #평가지표
                                     cv = 5)                       # 교차 검증 분할 수 

In [ ]:
log_y = np.log(y)  #타깃값 로그변환
gridsearch_ridge_model.fit(X_train, log_y) # 훈련

print('최적 하이퍼파라미터:', gridsearch_ridge_model.best_params_)

In [ ]:
preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

print(f'릿지 회귀 RMSLE 값: {rmsle(log_y, preds, True):.4f}')

In [ ]:
ridge_preds = gridsearch_ridge_model.predict(X_test)  #테스트 데이터로 예측

submission['count'] = np.exp(ridge_preds) #지수변환
submission.to_csv('submission_ridge.csv', index = False) #파일저장

# 라쏘 회귀모델 

In [ ]:
# 라쏘 모델훈련
from sklearn.linear_model import Lasso

# 모델생성
lasso_model = Lasso()

# 하이퍼파라미터 값 목록
lasso_alpha = 1/np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
lasso_params = {'max_iter':[3000], 'alpha': lasso_alpha}

# 그리드 서치 with rasso 객체 생성
gridsearch_lasso_model = GridSearchCV(estimator = lasso_model,
                                     param_grid = lasso_params,
                                     scoring = rmsle_scorer,
                                     cv = 5)

# 그리드서치 수행
log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_lasso_model.best_params_)

In [ ]:
# 라쏘 성능검증
# 예측
preds = gridsearch_lasso_model.best_estimator_.predict(X_train)

# 평가
print(f'라쏘 회귀 RMSLE 값 : {rmsle(log_y, preds, True):.4f}')


In [ ]:
lasso_preds = gridsearch_lasso_model.predict(X_test)  #테스트 데이터로 예측

submission['count'] = np.exp(lasso_preds) #지수변환
submission.to_csv('submission_lasso.csv', index = False) #파일저장

# 랜덤 포레스트 회귀 모델

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 모델 생성
randomforest_model = RandomForestRegressor()

#그리드서치 객체 생성
rf_params = {'random_state':[42], 'n_estimators': [100,120,140]} # random_state- 랜덤시드값, n_estimators- 결정트리개수
gridsearch_random_forest_model = GridSearchCV(estimator = randomforest_model,
                                             param_grid = rf_params,
                                             scoring = rmsle_scorer,
                                             cv = 5)
# 그리드서치 수행
log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)
print('최적 하이퍼파라미터 : ', gridsearch_random_forest_model.best_params_)


In [ ]:
# 예측
preds = gridsearch_random_forest_model.best_estimator_.predict(X_train)

# 평가
print(f'랜덤 포레스트 회귀 RMSLE 값 : {rmsle(log_y, preds, True):.4f}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

randomforest_preds = gridsearch_random_forest_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols=2)
figure.set_size_inches(12,4)

sns.histplot(y, bins=50, ax=axes[0])
axes[0].set_title('train data distribution')
sns.histplot(np.exp(randomforest_preds), bins = 50, ax=axes[1])
axes[1].set_title('predicted test data distribution');


In [ ]:
submission['count'] =np.exp(randomforest_preds)
submission.to_csv('submission_randomforest.csv', index=False)

reference

https://www.kaggle.com/code/viveksrinivasan/eda-ensemble-model-top-10-percentile/notebook
https://www.kaggle.com/code/kwonyoung234/for-beginner/notebook

https://docs.python.org/3/library/datetime.html
https://docs.python.org/ko/3/library/stdtypes.html?highlight=split

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


나중에 피쳐값 바꿔볼것들.
1. month를 지우지않고 해보기\
2. month 묶어서 seasson으로 만들때 다르게 해보기 
3. 